# Feature Extraction - Cardiac Phase Segmentation
This script is segmentating the records into their cardiac phases and extracts statistical features such as mean and standard deviation of a cycle. 

The processing is as follows:
1. PCG records are loaded and chunked.
2. Chunks are normalized usng RMS normalization
3. Peak detection algorithm is performed on records that were not needed to be manually corrected
4. Segmentation results of manually-corrected chunks are loaded
5. Segmentation results of all chunks are saved into a single numpy file for easy access.

**Note:** *This feature extraction could be also done at the stage of classification however, saving all the features once and accessing them repeatedly by means of loading it is saving time as compared to re-extracting the features.*

In [7]:
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from python_speech_features import mfcc
from python_speech_features import delta
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
import scipy
from utils import *
import random
import heartpy as hp
from scipy.signal import hilbert, cheby1, filtfilt
import numpy as np
from bokeh.plotting import figure, output_file, show
import wfdb
from segmentation_utils import *


**Data Loading and RMS Normalization**

In [8]:
sampling_frequency = 2000 # in Hz
slice_length = 4          # in seconds
overlap = 2               # in seconds

normal_records, normal_records_denoised = load_records(path = '../dataset/normal_tracings.txt',normalize = False, crop = 20000)
abnormal_records, abnormal_records_denoised = load_records(path = '../dataset/abnormal_tracings.txt',normalize = False, crop = 20000)
records_train = np.concatenate((abnormal_records_denoised, normal_records_denoised), axis=0)
labels = np.concatenate((np.ones((abnormal_records_denoised.shape[0],1)),np.zeros((normal_records.shape[0],1))), axis=0)

normal_records_chunks = np.zeros((normal_records_denoised.shape[0], 4, slice_length*sampling_frequency))
slices = np.arange(0, 10, slice_length-overlap, dtype=int)
for i in range(normal_records.shape[0]):
    j = 0
    for start, end in zip(slices[:-1], slices[1:]):
        start_audio = start * sampling_frequency
        end_audio = (end + overlap)* sampling_frequency
        chunk = normal_records_denoised[i, int(start_audio): int(end_audio)]
        normal_records_chunks[i, j, :] = chunk
        j = j + 1  
        
abnormal_records_chunks = np.zeros((abnormal_records_denoised.shape[0], 4, slice_length*sampling_frequency))
for i in range(abnormal_records.shape[0]):
    j = 0
    for start, end in zip(slices[:-1], slices[1:]):
        start_audio = start * sampling_frequency
        end_audio = (end + overlap)* sampling_frequency
        chunk = abnormal_records_denoised[i, int(start_audio): int(end_audio)]
        abnormal_records_chunks[i, j, :] = chunk
        j = j + 1  

abnormal_records_chunks_reshaped = np.reshape(abnormal_records_chunks, (33*4, 8000))
normal_records_chunks_reshaped = np.reshape(normal_records_chunks, (29*4, 8000))
all_records_chunked = np.concatenate((abnormal_records_chunks_reshaped, normal_records_chunks_reshaped), axis = 0)
all_records_chunked_normalized = np.zeros_like(all_records_chunked)
labels_chunked = np.concatenate((np.ones((abnormal_records_chunks_reshaped.shape[0],1)),np.zeros((normal_records_chunks_reshaped.shape[0],1))), axis=0)


for i in range(all_records_chunked.shape[0]):
    data = all_records_chunked[i,:]
    rms_level = 0
    r = 10**(rms_level / 10.0)
    a = np.sqrt( (len(data) * r**2) / np.sum(data**2) )
    data = data * a
    all_records_chunked_normalized[i,:] = data




**Feature Extraction**

In [9]:
all_mean_S1 = np.zeros((all_records_chunked_normalized.shape[0],1))
all_mean_S2 = np.zeros((all_records_chunked_normalized.shape[0],1))
all_std_S1 = np.zeros((all_records_chunked_normalized.shape[0],1))
all_std_S2 = np.zeros((all_records_chunked_normalized.shape[0],1))

all = np.arange(0,248,1)

corrected = [9,10,20,21,22,23,28,34,57,60,65,66,67,
                80,84,85,86,97,101,102,105,106,112,119,
                128,129,130,131,162,164,165,166,178,179,
                187,194,206,211,228,229,230,231,235,238,
                239,240,241,243,68,69,70,71,99,108,110,
                111,116,234]  # indices of manually corrected chunks

rejection = [68,69,70,71,108,110,111]  # indices of chunks that could not been segmented

all_clear = np.delete(all,corrected)

for j in range(all_mean_S1.shape[0]):
    
    if j not in corrected:
            
        PCG = all_records_chunked_normalized[j,:]
        sampling_rate = 2000

        PCG_f = cheby1_bandpass_filter(PCG, lowcut=10, highcut=500, fs=sampling_rate, order=4)
        dn = (np.append(PCG_f[1:], 0) - PCG_f)
        dtn = dn/(np.max(abs(dn)))
        an = abs(dtn)
        en = an**2
        sen = -abs(dtn) * np.log10(abs(dtn))
        sn = -(dtn**2) * np.log10(dtn**2)
        window_len = 50
        sn_f = np.insert(running_mean(sn, window_len), 0, [0] * (window_len - 1))
        zn = np.imag(hilbert(sn_f))
        ma_len = 4000
        zn_ma = np.insert(running_mean(zn, ma_len), 0, [0] * (ma_len - 1))
        zn_ma_s = zn - zn_ma

        idx = np.argwhere(np.diff(np.sign(zn_ma_s)) > 0).flatten().tolist()
        idx_search = []
        id_maxes = np.empty(0, dtype=int)
        search_window_half = round(sampling_rate * .15)  
        for i in idx:
            lows = np.arange(i-search_window_half, i)
            highs = np.arange(i+1, i+search_window_half+1)
            if highs[-1] > len(PCG):
                highs = np.delete(highs, np.arange(np.where(highs == len(PCG))[0], len(highs))) 
            PCG_window = np.concatenate((lows, [i], highs))
            idx_search.append(PCG_window)
            PCG_window_wave = PCG[PCG_window]
            id_maxes = np.append(id_maxes, PCG_window[np.where(PCG_window_wave == np.max(PCG_window_wave))[0]])

        id_maxes =  id_maxes[id_maxes >= 0]  # removal of potential negative indices
        id_maxes = np.unique(id_maxes)       # removal of potential duplicates
        _, ev,od = interval(id_maxes)
        min_int = np.minimum(np.mean(ev),np.mean(od))
        
        id_maxes =  clear_adjacents(id_maxes, min_int) # cleaning adjacent peaks that happened too soon (id_maxes)
        _, ev,od = interval(id_maxes)

        # depending on which set of intervals are longer, they are assigned either to S1 or S2    
        if np.mean(ev) > np.mean(od):
            meanS2 = np.mean(ev);
            stdS2 = np.std(ev); 
            meanS1 = np.mean(od);
            stdS1 = np.std(od);
        else:      
            meanS2 = np.mean(od);
            stdS2 = np.std(od); 
            meanS1 = np.mean(ev);
            stdS1 = np.std(ev);
        
        all_mean_S1[j] = meanS1 
        all_mean_S2[j] = meanS2 
        all_std_S1[j] = stdS1
        all_std_S2[j] = stdS2

        
    if j in corrected and j not in rejection:
        directory =  "cardiac_cycle_segmentation_features/corrected_chunks/chunk_" + str(j) + ".npy"
        values = np.load(directory)
        meanS1 = values[0,0]
        meanS2 = values[0,1]
        stdS1 = values[0,2]
        stdS2 = values[0,3]
        all_mean_S1[j] = meanS1 
        all_mean_S2[j] = meanS2 
        all_std_S1[j] = stdS1
        all_std_S2[j] = stdS2
        

**Saving segmentation features of all chunks**

In [4]:
all_mean_S1 = np.array(all_mean_S1)
all_mean_S2 = np.array(all_mean_S2)
all_std_S1 = np.array(all_std_S1)
all_std_S2 = np.array(all_std_S2)
ratio = all_mean_S1 / all_mean_S2

np.save("all_mean_S1.npy", all_mean_S1)
np.save("all_mean_S2.npy", all_mean_S2)
np.save("all_std_S1.npy", all_std_S1)
np.save("all_std_S2.npy", all_std_S2) 


